In [1]:
# 1º Step: Obtain data from PATH_DATA and load it into a DataFrame.

import pandas as pd 
import numpy as np

PATH_DATA = 'https://www.ispdados.rj.gov.br/Arquivos/BaseDPEvolucaoMensalCisp.csv'

try:
    df_robs = pd.read_csv(PATH_DATA, sep=';', encoding='iso-8859-1')
    print(f'Data obtained from PATH_DATA sucessfully. DataFrame loaded with {df_robs.shape[0]} rows and {df_robs.shape[1]} columns.')

except Exception as e:
    print(f'Error while obtaining data from PATH_DATA. {e}')


Data obtained from PATH_DATA sucessfully. DataFrame loaded with 36355 rows and 63 columns.


In [13]:
# 2º Step: Dellimiting and groupingby DataFrame.

try:
    data_correction = df_robs['regiao'].str.startswith('Grande Niter', na=False)
    df_robs.loc[data_correction, 'regiao'] = 'Grande Niterói'
    df_robs = df_robs[['cisp', 'regiao', 'munic', 'roubo_veiculo']]
    df_robs_gb = df_robs.groupby(['cisp', 'regiao', 'munic']).sum(['roubo_veiculo']).reset_index()
    df_robs_gb_filtered = df_robs_gb.sort_values(by='munic', ascending=True).reset_index()
    print(f'DataFrame grouped by by cisp, regiao, and munic with {df_robs_gb_filtered.shape[0]} rows and {df_robs_gb_filtered.shape[1]} columns.')

except Exception as e:
    print(f'Error while dellimiting and groupingby DataFrame. {e}')

DataFrame grouped by by cisp, regiao, and munic with 146 rows and 5 columns.


In [14]:
# 3º Step: Write all variables from a selected column into a archive .txt.

try:
    with open('municipalities.txt', 'w', encoding='utf-8-sig') as municipalities:
        municipalities.write('Municipalities:\n\n')
        municipalities.write('\n'.join(df_robs_gb_filtered['munic'].unique()))
    print('Municipalities written to municipalities.txt successfully.')

except Exception as e:
    print(f'Error while trying to write municipalities to municipalities.txt. {e}')

Municipalities written to municipalities.txt successfully.


In [15]:
# 4º Step: Write all variables from dataframe into a archive .txt.

try:
    with open('variables.txt', 'w', encoding='iso-8859-1') as titles:
        titles.write('Variables:\n\n')
        titles.write('\n'.join(df_robs_gb_filtered.head(0)))
    print('Variables written to variables.txt successfully.')

except Exception as e:
    print(f'Error while trying to write variables to variables.txt. {e}')

Variables written to variables.txt successfully.


In [16]:
# 5º Step: Obtain measures of central tendency.

try:
    array_robs = np.array(df_robs_gb_filtered['roubo_veiculo'])
    total = np.sum(array_robs)
    mean = np.mean(array_robs)
    median = np.median(array_robs)
    min = np.min(array_robs)
    max = np.max(array_robs)

    print('Measures of central tendency:'+'\n')
    print(f'Total: {total:.2f}')
    print(f'Mean: {mean:.2f}')
    print(f'Median: {median:.2f}')
    print(f'Min: {min:.2f}')
    print(f'Max: {max:.2f}')

except Exception as e:
    print(f'Error while obtaining measures of central tendency. {e}')

Measures of central tendency:

Total: 697681.00
Mean: 4778.64
Median: 928.00
Min: 2.00
Max: 36274.00


In [17]:
# 6º Step: Obtain measures of dispersion.

try:
    std_dev = np.std(array_robs)
    var = np.var(array_robs)
    q1 = np.percentile(array_robs, 25)
    q3 = np.percentile(array_robs, 75)
    iqr = q3 - q1

    print('Measures of dispersion:'+'\n')
    print(f'Standard Deviation: {std_dev:.2f}')
    print(f'Variance: {var:.2f}')
    print(f'Q1: {q1:.2f}')
    print(f'Q3: {q3:.2f}')
    print(f'IQR: {iqr:.2f}')

except Exception as e:
    print(f'Error while obtaining measures of dispersion. {e}')

Measures of dispersion:

Standard Deviation: 7359.23
Variance: 54158285.88
Q1: 91.50
Q3: 7262.50
IQR: 7171.00


In [30]:
# 7º Step: Obtain mayors and minors.

try:
    df_robs_mayors =  df_robs_gb_filtered[df_robs_gb_filtered['roubo_veiculo'] > q3].copy()
    df_robs_mayors['flag'] = 'mayors'

    df_robs_minors = df_robs_gb_filtered[df_robs_gb_filtered['roubo_veiculo'] < q1].copy()
    df_robs_minors['flag'] = 'minors'

    df_robs_flags = pd.concat([df_robs_mayors, df_robs_minors], ignore_index=True)

    print('Mayors and minors obtained from DataFrame successfully. Printing results below:')
    display(df_robs_flags)

except Exception as e:
    print(f'Error while obtaining mayors and minors: {e}')


Mayors and minors obtained from DataFrame successfully. Printing results below:


,index,cisp,regiao,munic,roubo_veiculo,flag
0,47,54,Baixada Fluminense,Belford Roxo,23967,mayors
1,54,60,Baixada Fluminense,Duque de Caxias,19513,mayors
2,53,59,Baixada Fluminense,Duque de Caxias,36274,mayors
3,56,62,Baixada Fluminense,Duque de Caxias,8667,mayors
4,46,53,Baixada Fluminense,Mesquita,8190,mayors
...,...,...,...,...,...,...
69,91,104,Interior,São José do Vale do Rio Preto,20,minors
70,137,155,Interior,São Sebastião do Alto,29,minors
71,139,157,Interior,Trajano de Morais,20,minors
72,79,91,Interior,Valença,49,minors


In [31]:
# 8º Step: Export processed dataframe to a .CSV file.

try:
    df_robs_flags.to_csv('tb_roubo_veiculos.csv', index=False)
    df_robs_flags.to_excel('tb_roubo_veiculos.xlsx', index=False)
    print("Processed DataFrame exported to 'tb_roubo_veiculos.csv' and 'tb_roubo_veiculos.xlsx' successfully.")

except Exception as e:
    print(f'Error while exporting processed DataFrame to a .CSV file. {e}')

Processed DataFrame exported to 'tb_roubo_veiculos.csv' and 'tb_roubo_veiculos.xlsx' successfully.


In [32]:
# 9º Step: Obtain data from PATH_DATA and load it into a DataFrame.

from sqlalchemy import create_engine, text

try:
    host = '127.0.0.1'
    user = 'root'
    password = ''
    database = 'bd_robs_flags'

    engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

    table = 'tb_roubo_veiculos'

    df_robs_flags.to_sql(table, con=engine, if_exists='append', index=False)
    print(f'DataFrame loaded into MySQL database {database} successfully.')


except Exception as e:
    print(f'Error while obtaining data from PATH_DATA. {e}')

DataFrame loaded into MySQL database bd_robs_flags successfully.


In [33]:
# 10º Step: Read and insert comand at 'conexao' in MySQL Workbench.

with engine.begin() as connection:
    comand_sql = text("SELECT * FROM tb_roubo_veiculos")
    result = connection.execute(comand_sql)
    for line in result:
        print(line)

with engine.begin() as connection:
    comand_sql = text(
        """
            INSERT INTO tb_roubo_veiculos (cisp, regiao, munic, roubo_veiculo, flag)
            VALUES (171, 'Grande Niterói', 'São Gonçalo', 500, 'mais')
        """
    )
    
    connection.execute(comand_sql)

(47, 54, 'Baixada Fluminense', 'Belford Roxo', 23967, 'mayors')
(54, 60, 'Baixada Fluminense', 'Duque de Caxias', 19513, 'mayors')
(53, 59, 'Baixada Fluminense', 'Duque de Caxias', 36274, 'mayors')
(56, 62, 'Baixada Fluminense', 'Duque de Caxias', 8667, 'mayors')
(46, 53, 'Baixada Fluminense', 'Mesquita', 8190, 'mayors')
(51, 57, 'Baixada Fluminense', 'Nilópolis', 12257, 'mayors')
(72, 78, 'Grande Niterói', 'Niterói', 8401, 'mayors')
(50, 56, 'Baixada Fluminense', 'Nova Iguaçu', 11522, 'mayors')
(45, 52, 'Baixada Fluminense', 'Nova Iguaçu', 15023, 'mayors')
(52, 58, 'Baixada Fluminense', 'Nova Iguaçu', 15657, 'mayors')
(25, 29, 'Capital', 'Rio de Janeiro', 15532, 'mayors')
(24, 28, 'Capital', 'Rio de Janeiro', 10771, 'mayors')
(23, 27, 'Capital', 'Rio de Janeiro', 26905, 'mayors')
(22, 26, 'Capital', 'Rio de Janeiro', 7518, 'mayors')
(21, 25, 'Capital', 'Rio de Janeiro', 10792, 'mayors')
(20, 24, 'Capital', 'Rio de Janeiro', 11418, 'mayors')
(19, 23, 'Capital', 'Rio de Janeiro', 10612,